In [ ]:
# Testing Cell
import aviary.api as av
from aviary.api import Settings
from aviary.utils.doctape import get_variable_name, glue_variable

str_problem_type = Settings.PROBLEM_TYPE
str_sizing = av.ProblemType.SIZING.value
str_alternate = av.ProblemType.ALTERNATE.value
str_fallout = av.ProblemType.FALLOUT.value

str_alternate_snippet = f'```\n{str_problem_type}, {str_alternate}\n```'
glue_variable('alternate_snippet', str_alternate_snippet, md_code=False)

fallout_snippet = f'```\n{str_problem_type}, {str_fallout}\n```'
glue_variable('fallout_snippet', fallout_snippet, md_code=False)

glue_variable(av.EquationsOfMotion.HEIGHT_ENERGY.value, md_code=False)
glue_variable(av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value, md_code=False)

AnalysisScheme = av.AnalysisScheme
str_collocation = get_variable_name(AnalysisScheme.COLLOCATION)
str_collocation = str_collocation.split('.')[1].lower()
glue_variable(str_collocation, md_code=False)
str_shooting = get_variable_name(AnalysisScheme.SHOOTING)
str_shooting = str_shooting.split('.')[1].lower()
glue_variable(str_shooting, md_code=False)

# Off-Design Missions

## Overview

Off-design missions are enabled for the following mission types:

* {glue:md}`height_energy` ({glue:md}`collocation`)    (completed)
* {glue:md}`2DOF` ({glue:md}`collocation`) (complete)
* {glue:md}`height_energy` ({glue:md}`shooting`)    (future work)
* {glue:md}`2DOF` ({glue:md}`shooting`)    (future work)

An off-design mission can be created either by directly entering the parameters of the sized aircraft or by using the built-in functionality to run an off-design in the same script as the sizing mission.
There are currently two types of off-design missions supported in Aviary; alternate and fallout.
For an alternate mission, the target range of the mission and the payload mass are specified and the fuel mass required is solved for.
Fallout missions meanwhile take the payload and fuel masses as inputs and determine the range of the aircraft.
To create a simple off-design mission, take your input deck and add the line

```{glue:md} fallout_snippet
:format: myst
```
or
```{glue:md} alternate_snippet
:format: myst
```

```{note}
You may need to revise some of the values in your input deck for the off-design mission.
Since the aircraft is not re-designed, it is assumed that provided inputs constitute a valid aircraft.
```

From there, the mission may be run as before.

In [ ]:
# Testing Cell
import aviary.api as av
from aviary.utils.doctape import glue_variable

file_path = av.get_path('examples/off_design_example.py').relative_to(av.top_dir.parent)
glue_variable(file_path, md_code=True)

You may also run multiple off-design missions in the same script used to solve the design mission.
An examples of this is shown in {glue:md}`aviary/examples/off_design_example.py`.

In [ ]:
# Testing Cell
import os

import aviary.api as av
from aviary.utils.doctape import check_contains

# make sure off_design_example.py exists in aviary/examples folder
off_design_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('off_design_example.py'),
    os.listdir(off_design_examples),
    error_string='{var} not in ' + str(off_design_examples),
    error_type=FileNotFoundError,
)